In [ ]:
import torch as th
import transformers as tr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os

In [ ]:
import sys
print(sys.version)

In [ ]:
datasets_path = '../Datasets/'
gdpr_path = datasets_path + 'gdpr4.csv'
gdpr = pd.read_csv(gdpr_path)
gdpr.head()

In [ ]:
gdpr.isna().sum()

In [ ]:
gdpr = gdpr.drop(columns=["Unnamed: 4", "Recitals"], axis=1)
gdpr.head()

In [ ]:
gdpr.isna().sum()

In [ ]:
gdpr["input_text"] = gdpr["Category"] + " " + gdpr["Description"] + " " + gdpr["GDPR Articles"]
gdpr.head()

In [ ]:
training_data = gdpr[["input_text"]]
training_data

In [ ]:
training_data_copy = training_data.copy()
training_data_copy['input_text'] = training_data_copy['input_text'].str.replace(r'\(\)', '', regex=True)
training_data_copy['input_text'] = training_data_copy['input_text'].str.replace(r'\s+', ' ', regex=True).str.strip()
training_data_copy

In [ ]:
training_data.to_csv('../Datasets/gdpr4_training_data.csv', index=False)
training_data.head()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_data, test_data = train_test_split(training_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

def tokenize_data(df: pd.DataFrame, tokenizer: tr.PreTrainedTokenizer, max_length: int = 128) -> pd.DataFrame:
    return tokenizer(
        df["input_text"].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encoding = tokenize_data(train_data, tokenizer)
val_encoding = tokenize_data(val_data, tokenizer)
test_encoding = tokenize_data(test_data, tokenizer)
train_encoding.keys()


In [ ]:
label_map = {label: idx for idx, label in enumerate(gdpr["Category"].unique())}

# Map the Category column to labels
train_data["labels"] = train_data["input_text"].apply(lambda x: label_map.get(x.split()[0], -1))
val_data["labels"] = val_data["input_text"].apply(lambda x: label_map.get(x.split()[0], -1))
test_data["labels"] = test_data["input_text"].apply(lambda x: label_map.get(x.split()[0], -1))

# Ensure there are no -1 labels (which indicate missing categories)
train_data = train_data[train_data["labels"] != -1]
val_data = val_data[val_data["labels"] != -1]
test_data = test_data[test_data["labels"] != -1]

# Convert labels to numpy arrays
training_labels = train_data["labels"].values
val_labels = val_data["labels"].values
test_data_labels = test_data["labels"].values


class TextClassificationDataSet(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: th.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = th.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
# Update the datasets with the new labels
train_dataset = TextClassificationDataSet(train_encoding, training_labels)
val_dataset = TextClassificationDataSet(val_encoding, val_labels)
test_dataset = TextClassificationDataSet(test_encoding, test_data_labels)


model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
)

# initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

In [ ]:
trainer.evaluate()